In [1]:
# Install necessary libraries
%pip install "pybamm[plot,cite]" -q

import pybamm
import matplotlib.pyplot as plt

# Define the battery model
model = pybamm.lithium_ion.DFN(
    {
        "SEI": "solvent-diffusion limited",
        "SEI porosity change": "true",
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",
        "particle mechanics": ("swelling and cracking", "swelling only"),
        "SEI on cracks": "true",
        "loss of active material": "stress-driven",
        "calculate discharge energy": "true",
    }
)

# Set model parameters
param = pybamm.ParameterValues("OKane2022")
var_pts = {
    "x_n": 5,
    "x_s": 5,
    "x_p": 5,
    "r_n": 30,
    "r_p": 30,
}

# Set target cycle count
target_cycle = 2000

print(f"Calculating cycle {target_cycle} ...")

# Define experiment protocol
exp_k = pybamm.Experiment(
    [
        "Hold at 4.2 V until C/100 (5 minute period)",
        "Rest for 4 hours (5 minute period)",
        "Discharge at 0.1C until 2.5 V (5 minute period)",
        "Charge at 0.3C until 4.2 V (5 minute period)",
        "Hold at 4.2 V until C/100 (5 minute period)",
    ]
    + [
        (
            "Discharge at 1C until 2.5 V",
            "Charge at 0.3C until 4.2 V (5 minute period)",
            "Hold at 4.2 V until C/100 (5 minute period)",
        )
    ]
    * target_cycle
    + ["Discharge at 0.1C until 2.5 V (5 minute period)"],
)

# Run simulation
sim_k = pybamm.Simulation(model, parameter_values=param, experiment=exp_k, var_pts=var_pts)
sol_k = sim_k.solve()

# Extract LLI loss (unit: A.h) and calculate remaining capacity
Q_LLI_final_k = sol_k["Total lithium lost [mol]"].entries[-1] * 96485.3 / 3600
remaining_capacity_k = (5 - Q_LLI_final_k) / 5 * 100

print(f"When k={target_cycle}, the remaining capacity is: {remaining_capacity_k:.2f}%")


Note: you may need to restart the kernel to use updated packages.
Calculating cycle 2000 ...


At t = 429.689 and h = 1.24737e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 189.689 and h = 9.66951e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 429.05 and h = 6.72501e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 189.05 and h = 2.1981e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 428.447 and h = 1.04866e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 188.447 and h = 6.4146e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 427.869 and h = 7.00392e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 187.864 and h = 3.03125e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 427.312 and h = 7.41622e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 187.312 and h = 7.06924e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t

When k=2000, the remaining capacity is: 93.83%
